# Time-dependent Schrödinger Equation

Consider the time-dependent one-dimensional quantum harmonic oscillator defined by the Hamiltonian:

$$
H = \frac{\hat{p}^2}{2m} + \frac{\omega^2}{2m} \left(\hat{q} - q_0(t)\right)^2
$$

where:
$$
q_0(t) = \frac{t}{T}, \quad t \in [0, T].
$$

The initial state is given by:
$$
|\Psi_0\rangle = |n = 0\rangle
$$
(the ground state of the harmonic oscillator).

Note: we will use the standard convention of $\hbar = m = 1$ .

## Objectives

1. Compute the time-evolved state $(|\Psi(t)\rangle)$ for different values of $(T)$.
2. Plot:
   - The square norm of $(|\Psi(t)\rangle)$ as a function of $(q)$ at different times.
   - The average position of the particle $(\langle q(t) \rangle)$ as a function of time $(t)$.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
from numpy.polynomial.hermite import hermval
from scipy.special import factorial

## Solution ##

The analytical expression for the eigenfunctions for this Hamiltonian can be expressed through the Hermite polynomials as follows:

$$
Ψ_n(q; t) = \left(\frac{ω}{π}\right)^{1/4}\frac{1}{\sqrt{2^nn!}}
            \exp\left({\frac{-ω(q - q₀(t))^2}{2}}\right)
            H_n\left(\sqrt{ω} (q - q₀(t))\right)
$$

So, first, let's define those.

In [ ]:
# Define Hermite polynomials
def hermite(x,n: int):
    
    # All coefficients of the Hermite polynomial set to zero except for the n-th, which is set to 1
    herm_coeff = np.zeros(n+1)
    herm_coeff[n] = 1
    
    # Compute the polynomial using the function from Scipy
    herm_pol = hermval(x,herm_coeff)
    
    return herm_pol

def decomposition(x, n, omega):
    ## Returns the eigenfunction of order n for the 1d quantum harmonic oscillator
    
    prefactor = (omega / np.pi)**(1/4) * (1 / np.sqrt(2**n * factorial(n)))
    psi = prefactor * np.exp(-omega * (x**2) / 2) * hermite(np.sqrt(omega) * x, n)
    
    return psi

# Objects inizialitation # 

Now, let's create classes for containing our operators and variables.

In [ ]:
class Param:
    ## Class for containing the parameters used in the simulation
    
    def __init__(self,
                 xmax: float,
                 Nx: int,
                 tsim: float,
                 Nt: int,
                 im_time: bool = False) -> None:
    
        self.xmax = xmax
        self.Nx = Nx
        self.tsim = tsim
        self.Nt = Nt
        self.im_time = im_time
        
        # Derived parameters
        self.dx = 2 * xmax/ Nx
        self.dt = tsim / Nt
        self.x = np.arange(-xmax + xmax / Nx,self.dx)
        # Momentum, defined as 1/x and a prefactor from Fourier Transfrom
        self.dk = np.pi / xmax
        # Momentum grid
        self.k = np.concatenate((np.arange(0, Nx / 2), np.arange(-Nx / 2, 0))) * self.dk
        
class Operators:   
    # This class will hold the operators values
    
    def __init__(self, 
                 res: int,
                 par: Param,
                 voffset: float = 0.,
                 wfcoffset: float = 0.,
                 q0 = None,
                 n: int = 0) -> None:

        # Initialize the containers for the various operators
        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)
        
        
        # Time-dependent offset (default to zero)
        self.q0 = q0 or (lambda q0: 0)
        
        # Initialize parameters and compute the energy
        if par is not None:
            self.init_operators()
            self.compute_energy()
            
    def init_operators(self, par: Param, voffset: float = 0, wfcoffset: float = 0, n: int = 0):
        
        
        
        
        